In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pcs.utils.torchutils import grad_reverse
from torch.autograd import Function
from torchvision import models
import pcs.utils.torchutils as torchutils

In [2]:
class CosineClassifier(nn.Module):
    def __init__(self, num_class=64, inc=4096, temp=0.05):
        super(CosineClassifier, self).__init__()
        self.fc = nn.Linear(inc, num_class, bias=False)
        self.num_class = num_class
        self.temp = temp

    def forward(self, x, reverse=False, eta=0.1):
        self.normalize_fc()

        if reverse:
            x = grad_reverse(x, eta)
        x = F.normalize(x)
        x_out = self.fc(x)
        x_out = x_out / self.temp

        return x_out

    def normalize_fc(self):
        self.fc.weight.data = F.normalize(self.fc.weight.data, p=2, eps=1e-12, dim=1)

    @torch.no_grad()
    def compute_discrepancy(self):
        self.normalize_fc()
        W = self.fc.weight.data
        D = torch.mm(W, W.transpose(0, 1))
        D_mask = 1 - torch.eye(self.num_class).cuda()
        return torch.sum(D * D_mask).item()


In [3]:
cls_head = CosineClassifier(num_class=30, inc=4096)
torchutils.weights_init(cls_head)
cls_head = cls_head.cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [4]:
torchutils.weights_init(cls_head)
cls_head = cls_head.cuda()

In [5]:
import segmentation_models_pytorch as smp
model = smp.Unet(encoder_name="resnet101")

In [9]:
from torchvision.datasets import Cityscapes
from torchvision import transforms
from pcs.utils.datautils_seg import *
from pcs.utils.segmentation_helper import *
from pcs.utils import (AverageMeter, datautils, is_div, per, reverse_domain,
                       torchutils, utils)


train_dataset = Cityscapes(root="./data/splits/cynthia/cityscapes", split="train", mode="coarse", transform=transforms.ToTensor(), target_type="color")
#train_dataset_targets = one_hot_masks(train_dataset)
#train_dataset_targets = down_sample_masks_dataset(train_dataset_targets)
#train_dataset.targets = train_dataset_targets

train_loader = datautils.create_loader(
                train_dataset,
                32,
                is_train=True,
                num_workers=16)

In [23]:
"""from PIL import Image
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __getitem__(self, index):
        img_path, label = self.data[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.data)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = MyDataset(train_dataset, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)"""

In [10]:
import torch

features = []
model.encoder.eval()

with torch.no_grad():
    for i, (images, targets) in enumerate(train_loader):
        # Pass images through the encoder
        encoder_features = model.encoder(images)
        # Store the features
        features.append(encoder_features)


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "/share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/collate.py", line 84, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/collate.py", line 84, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.7/site-packages/torch/utils/data/_utils/collate.py", line 86, in default_collate
   

In [5]:
temp = np.load("dick.npy", allow_pickle="TRUE")
print(temp)

{(0, 0, 0): 0, (70, 70, 70): 1, (180, 130, 70): 2, (0, 220, 220): 3, (35, 142, 107): 4, (153, 153, 153): 5, (30, 170, 250): 6, (70, 0, 0): 7, (142, 0, 0): 8, (0, 0, 255): 9, (32, 11, 119): 10, (232, 35, 244): 11, (128, 64, 128): 12, (60, 20, 220): 13, (156, 102, 102): 14, (153, 153, 190): 15, (100, 60, 0): 16, (230, 0, 0): 17, (152, 251, 152): 18, (100, 80, 0): 19, (160, 170, 250): 20, (110, 0, 0): 21, (140, 150, 230): 22, (180, 165, 180): 23, (81, 0, 81): 24, (90, 0, 0): 25, (100, 100, 150): 26, (90, 120, 150): 27}


In [25]:
############## Fresh try

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pcs.utils.torchutils import grad_reverse
from torch.autograd import Function
import torchvision
from torchvision import models
import pcs.utils.torchutils as torchutils
from torchvision.datasets import Cityscapes
from torchvision import transforms
from pcs.utils.datautils_seg import *
from pcs.utils.segmentation_helper import *
from pcs.utils import (AverageMeter, datautils, is_div, per, reverse_domain,
                       torchutils, utils)
import segmentation_models_pytorch as smp
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.Cityscapes(root='./data/splits/cynthia/cityscapes', split='train', mode='coarse', target_type='color',
                                     transform=torchvision.transforms.Compose([
                                         torchvision.transforms.ToTensor(),
                                         torchvision.transforms.Normalize(
                                             mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
                                     ])),
    batch_size=4, shuffle=True, num_workers=15)'''

#dataset = Cityscapes('./PCS-FUDA-v3/PCS-FUDA-master/data/splits/cynthia/cityscapes', split='train', mode='coarse',
#                     target_type='color')

train_dataset = Cityscapes(root="/projectnb/cs585bp/gmharsh/PCS-FUDA-v3/PCS-FUDA-master/data/splits/cynthia/cityscapes", 
                           split="train", 
                           mode="coarse", transform=image_transform, target_type="color", target_transform=mask_transform)

#train_dataset = Cityscapes(root="/projectnb/cs585bp/gmharsh/PCS-FUDA-v3/PCS-FUDA-master/data/splits/cynthia/cityscapes", 
#                           split="train", 
#                           mode="coarse", transform=torchvision.transforms.Compose([
#                                         torchvision.transforms.ToTensor(),
#                                         torchvision.transforms.Normalize(
#                                             mean=[0.485, 0.456, 0.406],
#                                             std=[0.229, 0.224, 0.225])]), target_type="color")

train_loader = datautils.create_loader(
                train_dataset,
                32,
                is_train=True,
                num_workers=15)

model = smp.Unet(
    encoder_name="resnet101",
    encoder_weights="imagenet",
    classes=30,
    activation=None,
).to(device)

In [2]:
def sparse_make_batch(labels_batch):
    result=[]
    for i in range(labels_batch.shape[0]):
        print("{}/{} ".format(i+1, labels_batch.shape[0]),)
        result.append(sparse_make_vectorized(labels_batch[i]))
        print("Result appended!")
    print("Out of sparse_make_batch loop... now creating torch.Tensor(result)")
    return torch.Tensor(result)

In [ ]:
for batch_i, (images, labels) in enumerate(train_loader):
    labels = sparse_make_batch(labels)
    print(labels.shape)
    print("Accessed one batch")
    break

1/32 
Result appended!
2/32 
Result appended!
3/32 
Result appended!
4/32 
Result appended!
5/32 
Result appended!
6/32 
Result appended!
7/32 
Result appended!
8/32 
Result appended!
9/32 
Result appended!
10/32 
Result appended!
11/32 
Result appended!
12/32 
Result appended!
13/32 
Result appended!
14/32 
Result appended!
15/32 
Result appended!
16/32 
Result appended!
17/32 
Result appended!
18/32 
Result appended!
19/32 
Result appended!
20/32 
Result appended!
21/32 
Result appended!
22/32 
Result appended!
23/32 
Result appended!
24/32 
Result appended!
25/32 
Result appended!
26/32 
Result appended!
27/32 
Result appended!
28/32 
Result appended!
29/32 
Result appended!
30/32 
Result appended!
31/32 
Result appended!
32/32 


In [3]:
for batch_i, (images, labels) in enumerate(train_loader):
    a = labels[30]
    b = labels[31]
    break

In [4]:
sparse_make_vectorized(labels[4])

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [2]:
features = []
model.encoder.eval()

ctr=0
with torch.no_grad():
    for images, masks in train_loader:
        ctr+=1
        print("Loading and storing feature: {}/{}".format(ctr, len(train_loader)))
        #images = images.to(device)
        #masks = masks.to(device)

        encoder_features = model.encoder(images)
        # Store the features
        features.append(encoder_features)
        

KeyboardInterrupt: 

In [5]:
import os
os.path.exists('labels.npy')

True

In [5]:
import torch
import segmentation_models_pytorch as smp
from torchvision.transforms import ToTensor, Normalize

# Define the encoder architecture
encoder = smp.Unet(
    encoder_name="resnet101",
    encoder_weights="imagenet",
    classes=19,
    activation=None,
).to(device)
encoder_channels = 512

# Define the pre-processing transforms
normalize_transform = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Load the Cityscapes dataset
dataset = Cityscapes('./data/splits/cynthia/cityscapes', split='train', mode='coarse', target_type='color')

# Initialize an empty tensor to store the feature maps for all images
batch_size = 16  # Change this to the batch size you want to use
features = torch.zeros((len(dataset), encoder_channels, 128, 256))

# Iterate over all images in the dataset and pass them through the encoder
for i in range(0, len(dataset), batch_size):
    print("Iteration: {}".format(i))
    # Get the current batch of images and segmentation masks
    batch_images, _ = zip(*[dataset[j] for j in range(i, min(i + batch_size, len(dataset)))])
    
    # Pre-process the images and convert them to a tensor
    batch_images = torch.stack([normalize_transform(ToTensor()(img)) for img in batch_images], dim=0)
    
    # Pass the images through the encoder
    batch_features = encoder(batch_images)
    
    # Store the feature maps for the current batch of images
    features[i:i + len(batch_images)] = batch_features

# Print the output shape
print(features.shape)


Iteration: 0


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor